In [1]:
import pandas as pd

import re
from pymorphy2 import MorphAnalyzer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import Parallel, delayed

from tqdm import tqdm
from joblib import Parallel, delayed

from IPython.display import display, Image

from math import sqrt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skliz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\skliz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('./data/data.csv')

In [3]:
def clean_text(x: str,
               morph: MorphAnalyzer = MorphAnalyzer(),
               sno: SnowballStemmer = SnowballStemmer('russian'),
               stop_words: list = stopwords.words('russian'),
               min_len_word: int = 2) -> str:
    # Приведение к нижнему регистру
    x = x.lower()
    # Удаление всех символов, цифр
    x = re.sub(r'[^а-я]+', ' ', x)
    # Удаление повторяющихся пробелов
    x = re.sub(r' +', ' ', x)

    word_tokenization = x.split(' ')   # Токенизация слов

    result_sentance = []

    # Приведение слов к нормальной форме (леммер + стеммер для причастий)
    for word in word_tokenization:
        if 'PRTF' in morph.parse(word.strip())[0].tag or 'PRTS' in morph.parse(word.strip())[0].tag:
            result_word = sno.stem(word)
        else:
            result_word = morph.parse(word.strip())[0].normal_form

        if result_word not in stop_words and len(result_word) > min_len_word:
            result_sentance.append(result_word)

    return ' '.join(result_sentance)

In [4]:
clean_description = Parallel(n_jobs=-1)(delayed(clean_text)(t)
                                            for t in tqdm(df['description']))
df['clean_description'] = clean_description

100%|██████████| 5769/5769 [01:29<00:00, 64.53it/s]


In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

In [6]:
X = vectorizer.fit_transform(df['clean_description'])
df['vectorizer'] = list(X)

In [11]:
def get_rec(title: tuple, df: pd.DataFrame, rating: tuple, poster_width: int = 700, num: int = 10) -> None:
    df_target = df.loc[df[f'title_{title[1]}'] == title[0]].reset_index(drop=True)
    
    if len(df_target):
        genres = '|'.join(df_target['genres'].values[0].split(','))
        
        df_neighbors = df.loc[(df[f'title_{title[1]}'] != title[0]) & (df['genres'].str.contains(genres)) & (df[f'{rating[1]}_rating'] >= rating[0])].reset_index(drop=True).copy()
        
        dists = []
        
        for i in tqdm(range(df_neighbors.shape[0])):
            dist = 0
            for x, y in zip(df_target['vectorizer'][0].toarray()[0], df_neighbors['vectorizer'][i].toarray()[0]):
                dist += (x - y)**2
            dists.append(sqrt(dist))
            
        df_neighbors['dists'] = dists
        
        print(f'Рекомендации для {title[0]}\n')
        
        results = df_neighbors.sort_values(by='dists', ascending=True)[:num].reset_index()
        
        for i in results.index:
            print(f"{i+1}. {results.iloc[i]['title_ru']} | {results.iloc[i]['title_eng']} ({results.iloc[i]['year_start']})")
            try:
                display(Image(url=results.iloc[i]['poster'], width=poster_width))
            except ValueError:
                print('Постер отсутствует')
            except TypeError:
                print('Постер отсутствует')
            print(f"Жанр: {results.iloc[i]['genres']}\nОписание: {results.iloc[i]['description']}")
            print(f"Myshows: {results.iloc[i]['myshows_rating']} ({results.iloc[i]['myshows_num']}) | KP: {results.iloc[i]['kp_rating']} ({results.iloc[i]['kp_num']}) | IMDB: {results.iloc[i]['imdb_rating']} ({results.iloc[i]['imdb_num']})\n\n")
        
    else:
        print('Упс! Такого еще нет в базе')

In [13]:
get_rec(
    title=('Ёлки', 'ru'),
    df=df,
    poster_width=500,
    rating=(6, 'kp'),
    num=5
)

100%|██████████| 1685/1685 [01:53<00:00, 14.91it/s]

Рекомендации для Ёлки

1. Ёлки 2 | nan (2011.0)


Жанр: Комедия,Романтика
Описание: За этот год много чего произошло, но наступит день, который всё расставит на свои места. Продолжение новогодних историй, которые происходили в жизни героев в фильме «Ёлки».
Myshows: 3.69 (27.0) | KP: 7.04 (139664) | IMDB: 6.2 (2408.0)


2. Нежность | nan (2018.0)


Жанр: Комедия,Комедия
Описание: Один день из жизни Елены Ивановны в Санкт-Петербурге в поисках мужчины, секса и, может быть, любви.
Myshows: nan (nan) | KP: 7.6 (6187) | IMDB: 7.4 (170.0)


3. Ёлки 3 | nan (2013.0)


Жанр: Комедия
Описание: Спустя два года они снова с нами: любимые герои «Ёлок» в самых невероятных новогодних историях. Боря и Женя, чьи годовалые дети в канун праздника доведут друзей до психушки. Маленькая девочка Настя, чьи родители разлучат ее влюбленных друг в друга собак. Лыжник и сноубордист в самой экстремальной в их жизни гонке — от военкома. И профессор из Екатеринбурга Андрей, чья любвеобильность вновь не доведет его до добра, а только до проруби в минус 30.  Под бой курантов этих и новых героев «Ёлок» объединит бумеранг добра. Завершая свой круг, он заслуженно вернётся к каждому, кто его однажды запустил — кому улыбками и объятиями любимых и друзей, кому — праздничным настроением и соответствующим градусу состоянием, а кому-то… телефоном в холодце!
Myshows: 3.38 (49.0) | KP: 6.49 (100680) | IMDB: 5.4 (1594.0)


4. Старый Новый год | New Year"s Eve (2011.0)


Жанр: Комедия,Романтика
Описание: Герои фильма — жители города, который никогда не спит. Нью-Йорк накануне Нового года переплетет жизни нескольких героев: умирающего в больнице пожилого человека, секретаршу, решившую осуществить данные себе обещания, продюсера знаменитого новогоднего шоу на Таймс Сквер, человека, который вообще ненавидит этот праздник.
Myshows: 4.23 (11.0) | KP: 6.86 (41847) | IMDB: 5.6 (85805.0)


5. Долгое падение | A Long Way Down (2014.0)


Жанр: Комедия,Драма
Описание: Четверо незнакомцев встречаются в канун Нового Года на крыше лондонского небоскреба. Каждый из них поднялся с одной целью — прыгнуть. Очередь в лучший мир заставляет их заключить пакт — дожить до Дня всех влюбленных и оторваться по полной.
Myshows: 3.67 (9.0) | KP: 6.57 (15520) | IMDB: 6.3 (33444.0)


